In [1]:
import pipeline.pipeline as pipeline
import db.knowhere_db as kdb
import time
import pandas as pd

In [ ]:
fName = r'C:\Users\William\Documents\DataScience\Projects\Knowhere\data\aggregated_emil_sensor2.csv'
csv_dir = r'C:\Users\William\Documents\DataScience\Projects\Knowhere\data\iPhone'

In [ ]:
user_df = pipeline.iphone(username='emil', file_with_path=fName, dir_to_aggregate=csv_dir)

In [ ]:
#writer = kdb.Writer('knowhere')
#writer.write_dataframe_to_collection(df=user_df, collection='iphone_test2')

In [3]:
reader = kdb.Reader('knowhere')
rdf = reader.get_dataframe_unrolled(collection='iphone_test',filter_args={'user_id': kdb.ObjectId('58c5b91d71a9e82f33ad1669')})
rdf[0:10]

sensor_name,Acceleration (x),Acceleration (y),Acceleration (z),Altimeter (Barometer) (Pressure),Altimeter (Barometer) (Relative Altitude),Attitude (Pitch),Attitude (Quarternion w),Attitude (Quarternion x),Attitude (Quarternion y),Attitude (Quarternion z),...,Memory (Inactive),Memory (Purgeable),Memory (Total),Memory (Used),Memory (Wired),Microphone (Left Channel Level),Microphone (Right Channel Level),Proximity (Active),WiFi (Connected BSSID),WiFi (Connected SSID)
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-03-09 08:09:29,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,false,None,None
2017-03-09 08:09:30,0.00816796813160181,-0.0082997465506196,0.0295017659664154,101.2510681152344,0,1.08533864966242,0.853545068960541,0.514771459259045,-0.0690450118436123,-0.041278884734921,...,None,None,None,None,None,None,None,None,None,None
2017-03-09 08:09:31,None,None,None,101.2524948120117,-0.1182122230529785,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2017-03-09 08:09:32,None,None,None,101.2539215087891,-0.237429141998291,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2017-03-09 08:09:33,None,None,None,101.2588119506836,-0.644869327545166,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2017-03-09 08:09:34,0.00862309336662292,0.00809421017765999,-0.0139299752190709,101.2613677978516,-0.8576369285583496,1.25981392171994,0.800085370094503,0.593825427241536,-0.0843774410652988,-0.0107335903316462,...,160358400.0,13934592.0,1037041664.0,752959488.0,204374016.0,-29.9767,-29.9767,None,Not Connected,Not Connected
2017-03-09 08:09:35,None,None,None,101.2613677978516,-0.8576369285583496,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2017-03-09 08:09:36,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2017-03-09 08:09:37,None,None,None,101.2613677978516,-0.8576369285583496,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
reader = kdb.Reader('knowhere')
rdf = reader.get_dataframe_pivoted(collection='iphone_test',filter_args={'user_id': kdb.ObjectId('58c5b91d71a9e82f33ad1669')})
rdf[0:10]